In [2]:
import sqlite3
import pandas as pd

# Connect to the Chinook database
conn = sqlite3.connect("Chinook_Sqlite.sqlite")





# 1. Data Familiarization

## Q1: Write an SQL query to list all table names in the database.

In [ ]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
df = pd.read_sql_query(query, conn)
print(df)

             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track


## Q2: Display the first 5 rows of the Customer table.

In [4]:

query = "SELECT * FROM Customer LIMIT 5;"
df = pd.read_sql_query(query, conn)
print(df)


   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                                            Company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                              None   
2                                              None   
3                                              None   
4                                  JetBrains s.r.o.   

                           Address                 City State         Country  \
0  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP          Brazil   
1          Theodor-Heuss-Straße 34            Stuttgart  None         Germany   
2                1498 rue Bélanger             Montréal    QC          Canada   
3                 Ullevålsveien 14                 Oslo  None          Norway   
4                

# 2. Sales Analysis

## Q3: Write a query to find the total sales per country. Order the results by the highest sales first.

In [ ]:
query = """
SELECT BillingCountry AS Country, SUM(Total) AS TotalSales
FROM Invoice
GROUP BY BillingCountry
ORDER BY TotalSales DESC;
"""
df = pd.read_sql_query(query, conn)
print(df)


           Country  TotalSales
0              USA      523.06
1           Canada      303.96
2           France      195.10
3           Brazil      190.10
4          Germany      156.48
5   United Kingdom      112.86
6   Czech Republic       90.24
7         Portugal       77.24
8            India       75.26
9            Chile       46.62
10         Ireland       45.62
11         Hungary       45.62
12         Austria       42.62
13         Finland       41.62
14     Netherlands       40.62
15          Norway       39.62
16          Sweden       38.62
17           Spain       37.62
18          Poland       37.62
19           Italy       37.62
20         Denmark       37.62
21         Belgium       37.62
22       Australia       37.62
23       Argentina       37.62


## Q4: Find the top 5 customers by total purchase amount. Display their full name and total amount.

In [ ]:

query = """
SELECT FirstName || ' ' || LastName AS FullName, SUM(Total) AS TotalAmount
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY TotalAmount DESC
LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
print(df)


             FullName  TotalAmount
0         Helena Holý        49.62
1  Richard Cunningham        47.62
2          Luis Rojas        46.62
3     Ladislav Kovács        45.62
4       Hugh O'Reilly        45.62


## Q5: Which genres are most purchased? Show the genre name and the number of purchases.
 

In [ ]:

query = """
SELECT Genre.Name AS GenreName, COUNT(*) AS PurchaseCount
FROM InvoiceLine
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Genre ON Track.GenreId = Genre.GenreId
GROUP BY Genre.GenreId
ORDER BY PurchaseCount DESC;
"""
df = pd.read_sql_query(query, conn)
print(df)


             GenreName  PurchaseCount
0                 Rock            835
1                Latin            386
2                Metal            264
3   Alternative & Punk            244
4                 Jazz             80
5                Blues             61
6             TV Shows             47
7            Classical             41
8             R&B/Soul             41
9               Reggae             30
10               Drama             29
11                 Pop             28
12    Sci Fi & Fantasy             20
13          Soundtrack             20
14         Hip Hop/Rap             17
15          Bossa Nova             15
16         Alternative             14
17               World             13
18   Electronica/Dance             12
19         Heavy Metal             12
20      Easy Listening             10
21              Comedy              9
22     Science Fiction              6
23       Rock And Roll              6


# 3. Employee & Customer Insights

## Q6: For each sales agent, show how many customers they manage and the total sales made by those customers.

In [ ]:

query = """
SELECT e.FirstName || ' ' || e.LastName AS SalesAgent,
       COUNT(DISTINCT c.CustomerId) AS CustomersManaged,
       ROUND(SUM(i.Total), 2) AS TotalSales
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId
WHERE e.Title = 'Sales Support Agent'
GROUP BY e.EmployeeId;
"""
df = pd.read_sql_query(query, conn)
print(df)


      SalesAgent  CustomersManaged  TotalSales
0   Jane Peacock                21      833.04
1  Margaret Park                20      775.40
2  Steve Johnson                18      720.16


## Q7: List the names of customers who have not made any purchases.

In [ ]:

query = """
SELECT FirstName || ' ' || LastName AS CustomerName
FROM Customer
WHERE CustomerId NOT IN (
    SELECT DISTINCT CustomerId FROM Invoice
);
"""
df = pd.read_sql_query(query, conn)
print(df)


Empty DataFrame
Columns: [CustomerName]
Index: []


# 4. Advanced Querying

## Q8: Write a query that returns the top 3 albums that generated the most revenue. Show album title and revenue

In [ ]:

query = """
SELECT Album.Title,
       ROUND(SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity), 2) AS Revenue
FROM InvoiceLine
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
GROUP BY Album.AlbumId
ORDER BY Revenue DESC
LIMIT 3;
"""
df = pd.read_sql_query(query, conn)
print(df)


                                      Title  Revenue
0  Battlestar Galactica (Classic), Season 1    35.82
1                      The Office, Season 3    31.84
2                            Minha Historia    26.73


## Q9: Find the average invoice total per customer and sort by the highest averages.

In [ ]:

query = """
SELECT FirstName || ' ' || LastName AS CustomerName,
       ROUND(AVG(Total), 2) AS AvgInvoiceTotal
FROM Customer
JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
GROUP BY Customer.CustomerId
ORDER BY AvgInvoiceTotal DESC;
"""
df = pd.read_sql_query(query, conn)
print(df)


             CustomerName  AvgInvoiceTotal
0             Helena Holý             7.09
1      Richard Cunningham             6.80
2              Luis Rojas             6.66
3         Ladislav Kovács             6.52
4           Hugh O'Reilly             6.52
5           Frank Ralston             6.23
6           Julia Barnett             6.23
7         Fynn Zimmermann             6.23
8         Puja Srivastava             6.11
9           Astrid Gruber             6.09
10         Victor Stevens             6.09
11       Terhi Hämäläinen             5.95
12  František Wichterlová             5.80
13       Isabelle Mercier             5.80
14  Johannes Van der Berg             5.80
15         Luís Gonçalves             5.66
16      François Tremblay             5.66
17           Bjørn Hansen             5.66
18             Jack Smith             5.66
19             Dan Miller             5.66
20        Heather Leacock             5.66
21         João Fernandes             5.66
22         